# Ensuring Consistency

**Activity Overview**: Ensure consistency by identifying and resolving conflicting values across datasets.

## Title: Customer Address Discrepancies

**Task**: Address customer address mismatches between CRM and marketing databases.

**Steps**:
1. Compare customer addresses in the CRM with those in the marketing database.
2. Identify records with conflicting address information.
3. Propose a method to consolidate records with verified addresses.

In [1]:
import pandas as pd

def find_address_discrepancies(crm_file, marketing_file, customer_id_col="customer_id", address_col="address"):
    """
    Compares customer addresses between CRM and marketing databases and identifies discrepancies.

    Args:
        crm_file (str): Path to the CSV file containing CRM customer data.
        marketing_file (str): Path to the CSV file containing marketing customer data.
        customer_id_col (str, optional): Name of the customer ID column. Defaults to "customer_id".
        address_col (str, optional): Name of the address column. Defaults to "address".

    Returns:
        pandas.DataFrame: A DataFrame containing customers with conflicting addresses,
                          or None if an error occurs.
    """
    try:
        # Read the CSV files into Pandas DataFrames
        crm_df = pd.read_csv(crm_file)
        marketing_df = pd.read_csv(marketing_file)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

    # Check if the required columns exist in both DataFrames
    if customer_id_col not in crm_df.columns or address_col not in crm_df.columns:
        print(f"Error: Missing required columns in CRM file. Expected '{customer_id_col}' and '{address_col}'.")
        return None
    if customer_id_col not in marketing_df.columns or address_col not in marketing_df.columns:
        print(f"Error: Missing required columns in marketing file. Expected '{customer_id_col}' and '{address_col}'.")
        return None

    # Merge the DataFrames on customer ID
    merged_df = pd.merge(crm_df, marketing_df, on=customer_id_col, suffixes=('_crm', '_marketing'))

    # Find records with conflicting address information
    conflicting_addresses_df = merged_df[merged_df[f'{address_col}_crm'] != merged_df[f'{address_col}_marketing']]

    return conflicting_addresses_df



def consolidate_addresses(conflicting_df, verified_source='crm'):
    """
    Consolidates customer addresses from conflicting records, preferring a verified source.

    Args:
        conflicting_df (pandas.DataFrame): A DataFrame containing customers with conflicting addresses,
                                            as returned by find_address_discrepancies.
        verified_source (str, optional): The preferred source for verified addresses ('crm' or 'marketing').
                                            Defaults to 'crm'.

    Returns:
        pandas.DataFrame: A DataFrame with consolidated addresses, or None if input is invalid.
    """
    if not isinstance(conflicting_df, pd.DataFrame) or conflicting_df.empty:
        print("Error: Input DataFrame is not valid or is empty.")
        return None

    if verified_source not in ('crm', 'marketing'):
        print("Error: Invalid verified_source. Must be 'crm' or 'marketing'.")
        return None

    address_col = 'address'
    consolidated_df = conflicting_df[[f'customer_id', f'{address_col}_crm', f'{address_col}_marketing']].copy()  # Create a copy to avoid modifying the original DataFrame

    # Consolidate addresses, preferring the verified source
    if verified_source == 'crm':
        consolidated_df['consolidated_address'] = consolidated_df.apply(
            lambda row: row[f'{address_col}_crm'] if pd.notnull(row[f'{address_col}_crm']) else row[f'{address_col}_marketing'], axis=1
        )
    else:  # verified_source == 'marketing'
        consolidated_df['consolidated_address'] = consolidated_df.apply(
            lambda row: row[f'{address_col}_marketing'] if pd.notnull(row[f'{address_col}_marketing']) else row[f'{address_col}_crm'], axis=1
        )

    return consolidated_df[['customer_id', 'consolidated_address']]



def main():
    """
    Main function to run the address discrepancy check and consolidation process.
    """
    # Provide the paths to your CSV files
    crm_file = 'crm_customers.csv'  # Replace with your actual file path
    marketing_file = 'marketing_customers.csv'  # Replace with your actual file path

    # Create dummy CSV files for demonstration
    try:
        with open(crm_file, 'w') as f:
            f.write("customer_id,address,name\n1,123 Main St,Alice\n2,456 Oak Ave,Bob\n3,789 Pine Ln,Charlie\n4,222 Elm St,David")
        with open(marketing_file, 'w') as f:
            f.write("customer_id,address,email\n1,123 Main St,alice@example.com\n2,456 Oak Ave,bob@example.com\n3,789 Pine Ln,charlie@example.com\n4,555 Maple Dr,david@example.com")
    except FileExistsError:
        pass

    # 1. Compare customer addresses
    conflicting_addresses_df = find_address_discrepancies(crm_file, marketing_file)

    # Print the results
    if conflicting_addresses_df is not None:
        if not conflicting_addresses_df.empty:
            print("Customer address discrepancies found:")
            print(conflicting_addresses_df.to_string(index=False))
        else:
            print("No customer address discrepancies found.")

        # 2. Propose a method to consolidate records with verified addresses.
        #    Example: Prefer addresses from the CRM database (you can change this).
        print("\nConsolidating addresses, preferring CRM data:")
        consolidated_df = consolidate_addresses(conflicting_addresses_df, verified_source='crm')
        if consolidated_df is not None:
            print(consolidated_df.to_string(index=False))
    else:
        print("An error occurred during address comparison. Please check the file paths and column names.")



if __name__ == "__main__":
    main()


Customer address discrepancies found:
 customer_id address_crm  name address_marketing             email
           4  222 Elm St David      555 Maple Dr david@example.com

Consolidating addresses, preferring CRM data:
 customer_id consolidated_address
           4           222 Elm St
